In [1]:
from pyspark import SparkContext, SparkConf, SQLContext

conf = SparkConf() \
    .setAppName("test_JDBC2") \
    .setMaster("spark://spark-master:7077") \
    .set("spark.jars", "/opt/workspace/jars/mysql-connector-java-8.0.21.jar")

sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

22/02/21 06:20:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark.read.format("jdbc") \
                .option("driver", "com.mysql.cj.jdbc.Driver") \
                .option("url", "jdbc:mysql://mysql2:3306/employee?serverTimezone=UTC&useSSL=false") \
                .option("dbtable", "dept_csv") \
                .option("user", "root") \
                .option("password", "1234") \
                .load()

In [3]:
df.show()

+----------+--------+------+
|     dname|     loc|deptno|
+----------+--------+------+
|ACCOUNTING|NEW YORK|    10|
|  RESEARCH|  DALLAS|    20|
|     SALES| CHICAGO|    30|
|OPERATIONS|  BOSTON|    40|
| MARKETING|   SUWON|    50|
|        IT| GWANGYO|    60|
|    HEALTH|   SEOUL|    70|
|      ARTS|   BUSAN|    80|
+----------+--------+------+



In [4]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

data = [("HISTORY", "SUWON", 50)]

schema = StructType([
    StructField("dname", StringType(),True),
    StructField("loc", StringType(),True),
    StructField("deptno", IntegerType(),True)
  ])
 
df_2 = spark.createDataFrame(data=data, schema=schema)
# df_2.printSchema()
df_2.show(truncate=False)

+-----+-----+------+
|dname|loc  |deptno|
+-----+-----+------+
|NURSE|SEOUL|90    |
+-----+-----+------+



In [5]:
df_2.write.format('jdbc') \
    .mode('append') \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", "jdbc:mysql://mysql2:3306/employee?serverTimezone=UTC&useSSL=false") \
    .option("dbtable", "dept_csv") \
    .option("user", "root") \
    .option("password", "1234") \
    .save()

In [9]:
sc.stop()
spark.stop()